In [1]:
import pandas as pd
import numpy as np
#########################
max_length =50
number_of_sequence=489348
sequence_size=max_length*4+1
number_of_element=number_of_sequence*sequence_size

######################################################################################

def make_image(seq):
    image=np.zeros(shape=(4,max_length),dtype=np.float16)
                 
    for i in range(len(seq)):
        if seq[i]=='A':
            image[0,i]=1
        elif seq[i]=='T':
            image[1,i]=1
        elif seq[i]=='G':
             image[2,i]=1
        else:
            image[3,i]=1
                            
    return image

######################################################################################

def encode_feature(seq,lable):
    s=np.zeros(shape=(201,1),dtype=np.float16)
    x_image=make_image(seq) 
    s[0]=lable
    s[1:201]=x_image.reshape(4*max_length,1)
    return s

######################################################################################

def Generate_binary(seq_file):
    s=np.zeros(shape=(number_of_element,1),dtype=np.float16)
    data=pd.read_csv(seq_file)
    seq=data['UTR'].values
    label=data['growth_rate'].values
    start=0
    end=sequence_size
    for i in range(number_of_sequence):
        e=encode_feature(data['UTR'].values[i],data['growth_rate'].values[i])
        s[start:end]=encode_feature(data['UTR'].values[i],data['growth_rate'].values[i])
        start=start+sequence_size
        end=end+sequence_size
        if(i%10000==0):
            print(i)  
    s.tofile("s.bin")

In [2]:
Generate_binary("Random_UTRs.csv") 

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000


In [3]:
import tensorflow as tf
import numpy as np
import os 

_HEIGHT = 4
_WIDTH = 50
_DEPTH = 1
NUM_CLASSES = 1
NUM_DATA_FILES = 1


filepath='C:\\Users\\H P\\nn_yeast'
filename='s.bin'

file_data= [os.path.join(filepath, filename)]

In [4]:
number_of_bytes_in_file_data=sum([os.path.getsize(os.path.join(filepath, filename))])
record_size=402

In [5]:
def get_filenames(is_training, data_dir):
    if is_training:
        return [os.path.join(data_dir, 's.bin')]
    else:
        return [os.path.join(data_dir, 's.bin')]

def record_dataset(filenames):
  """Returns an input pipeline Dataset from `filenames`."""
  record_bytes = 402
  return tf.data.FixedLengthRecordDataset(filenames, record_bytes)

def parse_record(raw_record):
  """Parse CIFAR-10 image and label from a raw record."""
  # Every record consists of a label followed by the image, with a fixed number
  # of bytes for each.
  label_bytes = 2
  image_bytes = 2*(_HEIGHT * _WIDTH * _DEPTH)
  record_bytes = label_bytes + image_bytes

  # Convert bytes to a vector of uint8 that is record_bytes long.
  record_vector = tf.decode_raw(raw_record, tf.float16)

  # The first byte represents the label, which we convert from uint8 to int32
  # and then to one-hot.
  label = tf.cast(record_vector[0], tf.float32)
  #label = tf.one_hot(label, _NUM_CLASSES)

  # The remaining bytes after the label represent the image, which we reshape
  # from [depth * height * width] to [depth, height, width].
  depth_major = tf.reshape(
      record_vector[1:201], [_DEPTH, _HEIGHT, _WIDTH])

  # Convert from [depth, height, width] to [height, width, depth], and cast as
  # float32.
  image = tf.cast(tf.transpose(depth_major, [1, 2, 0]), tf.float32)

  return image, label



def input_fn(is_training, data_dir, batch_size, num_epochs=1):
  """Input_fn using the tf.data input pipeline for CIFAR-10 dataset.

  Args:
    is_training: A boolean denoting whether the input is for training.
    data_dir: The directory containing the input data.
    batch_size: The number of samples per batch.
    num_epochs: The number of epochs to repeat the dataset.

  Returns:
    A tuple of images and labels.
  """
  dataset = record_dataset(get_filenames(is_training, data_dir))

  if is_training:
    # When choosing shuffle buffer sizes, larger sizes result in better
    # randomness, while smaller sizes have better performance. Because CIFAR-10
    # is a relatively small dataset, we choose to shuffle the full epoch.
    dataset = dataset.shuffle(buffer_size=5000)

  dataset = dataset.map(parse_record)
  
  dataset = dataset.prefetch(2 * batch_size)

  # We call repeat after shuffling, rather than before, to prevent separate
  # epochs from blending together.
  dataset = dataset.repeat(num_epochs)

  # Batch results by up to batch_size, and then fetch the tuple from the
  # iterator.
  dataset = dataset.batch(batch_size)
  iterator = dataset.make_one_shot_iterator()
  images, labels = iterator.get_next()

  return images, labels

(seq,label)=input_fn(1, filepath, 100000, num_epochs=1)

with tf.Session() as sess:
    for i in range(1):
        seq_example=sess.run(seq)
        print(seq_example)


[[[[1.]
   [1.]
   [1.]
   ...
   [1.]
   [0.]
   [1.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [1.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]]


 [[[1.]
   [1.]
   [1.]
   ...
   [0.]
   [0.]
   [1.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [1.]
   [1.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]]


 [[[1.]
   [1.]
   [1.]
   ...
   [0.]
   [1.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [1.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [1.]]]


 ...


 [[[1.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [1.]]

  [[0.]
   [0.]
   [1.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [1.]
   [0.]
   ...
   [1.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [1.]
   [0.]]]


 [[[1.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]


In [6]:
_BATCH_NORM_DECAY = 0.997
_BATCH_NORM_EPSILON = 1e-5

def batch_norm_relu(inputs, is_training, data_format):
  """Performs a batch normalization followed by a ReLU."""
  # We set fused=True for a significant performance boost. See
  # https://www.tensorflow.org/performance/performance_guide#common_fused_ops
  inputs = tf.layers.batch_normalization(
      inputs=inputs, axis=1 if data_format == 'channels_first' else 3,
      momentum=_BATCH_NORM_DECAY, epsilon=_BATCH_NORM_EPSILON, center=True,
      scale=True, training=is_training, fused=True)
  inputs = tf.nn.relu(inputs)
  return inputs

def conv2d_fixed_padding(inputs, filters, kernel_size, strides, data_format):
  """Strided 2-D convolution with explicit padding."""
  # The padding is consistent and is based only on `kernel_size`, not on the
  # dimensions of `inputs` (as opposed to using `tf.layers.conv2d` alone).
  if strides > 1:
    inputs = fixed_padding(inputs, kernel_size, data_format)

  return tf.layers.conv2d(
      inputs=inputs, filters=filters, kernel_size=kernel_size, strides=strides,
      padding='SAME', use_bias=False,
      kernel_initializer=tf.variance_scaling_initializer(),
      data_format=data_format)

def building_block(inputs, filters, is_training, projection_shortcut, strides,
                   data_format):
  """Standard building block for residual networks with BN before convolutions.

  Args:
    inputs: A tensor of size [batch, channels, height_in, width_in] or
      [batch, height_in, width_in, channels] depending on data_format.
    filters: The number of filters for the convolutions.
    is_training: A Boolean for whether the model is in training or inference
      mode. Needed for batch normalization.
    projection_shortcut: The function to use for projection shortcuts (typically
      a 1x1 convolution when downsampling the input).
    strides: The block's stride. If greater than 1, this block will ultimately
      downsample the input.
    data_format: The input format ('channels_last' or 'channels_first').

  Returns:
    The output tensor of the block.
  """
  shortcut = inputs
  inputs = batch_norm_relu(inputs, is_training, data_format)

  # The projection shortcut should come after the first batch norm and ReLU
  # since it performs a 1x1 convolution.
  if projection_shortcut is not None:
    shortcut = projection_shortcut(inputs)

  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=2, strides=strides,
      data_format=data_format)

  inputs = batch_norm_relu(inputs, is_training, data_format)
  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=2, strides=1,
      data_format=data_format)

  return inputs + shortcut

def block_layer(inputs, filters, block_fn, blocks, strides, is_training, name,
                data_format):
  """Creates one layer of blocks for the ResNet model.

  Args:
    inputs: A tensor of size [batch, channels, height_in, width_in] or
      [batch, height_in, width_in, channels] depending on data_format.
    filters: The number of filters for the first convolution of the layer.
    block_fn: The block to use within the model, either `building_block` or
      `bottleneck_block`.
    blocks: The number of blocks contained in the layer.
    strides: The stride to use for the first convolution of the layer. If
      greater than 1, this layer will ultimately downsample the input.
    is_training: Either True or False, whether we are currently training the
      model. Needed for batch norm.
    name: A string name for the tensor output of the block layer.
    data_format: The input format ('channels_last' or 'channels_first').

  Returns:
    The output tensor of the block layer.
  """
  # Bottleneck blocks end with 4x the number of filters as they start with
  filters_out = 4 * filters if block_fn is bottleneck_block else filters

  def projection_shortcut(inputs):
    return conv2d_fixed_padding(
        inputs=inputs, filters=filters_out, kernel_size=1, strides=strides,
        data_format=data_format)

  # Only the first block per block_layer uses projection_shortcut and strides
  inputs = block_fn(inputs, filters, is_training, projection_shortcut, strides,
                    data_format)


  for _ in range(1, blocks):
    inputs = block_fn(inputs, filters, is_training, None, 1, data_format)

  return tf.identity(inputs, name)

def bottleneck_block(inputs, filters, is_training, projection_shortcut,
                     strides, data_format):
  """Bottleneck block variant for residual networks with BN before convolutions.

  Args:
    inputs: A tensor of size [batch, channels, height_in, width_in] or
      [batch, height_in, width_in, channels] depending on data_format.
    filters: The number of filters for the first two convolutions. Note that the
      third and final convolution will use 4 times as many filters.
    is_training: A Boolean for whether the model is in training or inference
      mode. Needed for batch normalization.
    projection_shortcut: The function to use for projection shortcuts (typically
      a 1x1 convolution when downsampling the input).
    strides: The block's stride. If greater than 1, this block will ultimately
      downsample the input.
    data_format: The input format ('channels_last' or 'channels_first').

  Returns:
    The output tensor of the block.
  """
  shortcut = inputs
  inputs = batch_norm_relu(inputs, is_training, data_format)

  # The projection shortcut should come after the first batch norm and ReLU
  # since it performs a 1x1 convolution.
  if projection_shortcut is not None:
    shortcut = projection_shortcut(inputs)

  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=1, strides=1,
      data_format=data_format)

  inputs = batch_norm_relu(inputs, is_training, data_format)
  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=filters, kernel_size=3, strides=strides,
      data_format=data_format)

  inputs = batch_norm_relu(inputs, is_training, data_format)
  inputs = conv2d_fixed_padding(
      inputs=inputs, filters=4 * filters, kernel_size=1, strides=1,
      data_format=data_format)

  return inputs + shortcut




def seqnet_resnet(inputs):
    seq_data_format =  'channels_last'
    num_blocks = 1
    is_training=True

    inputs = conv2d_fixed_padding(inputs=inputs, filters=4, kernel_size=3, strides=1,data_format=seq_data_format)
    inputs = tf.identity(inputs, 'initial_conv')

    inputs = block_layer(inputs=inputs, filters=4, block_fn=building_block, blocks=num_blocks,
             strides=1, is_training=is_training, name='block_layer1',
             data_format=seq_data_format)

    inputs = block_layer(inputs=inputs, filters=4, block_fn=building_block, blocks=num_blocks,
             strides=1, is_training=is_training, name='block_layer2',
             data_format=seq_data_format)

    inputs = block_layer(inputs=inputs, filters=4, block_fn=building_block, blocks=num_blocks,
             strides=1, is_training=is_training, name='block_layer3',
             data_format=seq_data_format)

    inputs = batch_norm_relu(inputs, is_training,data_format=seq_data_format)

    inputs = tf.layers.average_pooling2d(
            inputs=inputs, pool_size=3, strides=1, padding='VALID',
            data_format=seq_data_format)
    inputs = tf.identity(inputs, 'final_avg_pool')

    inputs = tf.reshape(inputs, [-1, 6*64])

    inputs = tf.layers.dense(inputs=inputs, units=16)
    inputs = tf.layers.dense(inputs=inputs, units=1)


    inputs = tf.identity(inputs, 'final_dense')
    
    return inputs

    return model


    
network = seqnet_resnet(seq)
labelT=tf.reshape(label,[-1,1])

loss = tf.losses.mean_squared_error(predictions=network, labels=labelT)

with tf.Session() as sess:
    for i in range(1):
        L1_output=sess.run(tf.global_variables_initializer())
        Network=sess.run(network)
        Loss=sess.run(loss)
        Label=sess.run(label)


        print(Network,Loss,Label)




[[ 0.19857812]
 [ 0.21863428]
 [ 0.18737426]
 ...
 [-0.01803339]
 [ 0.37763187]
 [-0.13725513]] 2.0660594 [ 1.1835938  -0.20300293  0.52001953 ... -1.2275391   0.52001953
  0.11206055]
